In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.getcwd()
os.chdir('/content/drive/MyDrive')
os.getcwd()

'/content/drive/MyDrive'

# Install the necessary packages to run this script

In [ ]:
!pip install modifiers.ModifiedRobertaWithAdaptersV5

ERROR: Could not find a version that satisfies the requirement modifiers.ModifiedRobertaWithAdaptersV5 (from versions: none)
ERROR: No matching distribution found for modifiers.ModifiedRobertaWithAdaptersV5


### Load the necessary packages to run the script

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, RobertaConfig
from datasets import load_dataset
import torch
from sklearn.metrics import accuracy_score
from modifiers.ModifiedRobertaWithAdaptersV5 import ModifiedRobertaForSequenceClassification


ModuleNotFoundError: No module named 'modifiers'

Now lets check your GPU availability and load some sanity checkers. By default you should be using your gpu for this assignment if you have one available.

In [ ]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cuda


## **1.2: Load Data**
Loading the imdb dataset

In [ ]:
# Load IMDb dataset
dataset = load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# load the tokenizer for Roberta
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
# define the function for tokenizing the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding = "max_length", truncation= True, max_length= 512)

# Apply the tokenizer to the datasets
tokenized_dataset = dataset.map(tokenize_function, batched= True)

# Set the format of the dataset to return PyTorch tensors
tokenized_datasets = tokenized_dataset.remove_columns(["text"])
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Split the dataset into train and test sets
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
small_train_dataset = train_dataset.select(range(1000))
small_test_dataset = test_dataset.select(range(100))

Setting the base model and the metric function to be used to evaluate the model

In [ ]:
# Load the pre-trained RoBERTa model
base_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
def compute_metrics(p):
    predictions, labels = p
    predictions = torch.argmax(torch.tensor(predictions), dim=-1)
    accuracy = accuracy_score(labels, predictions)
    return {'accuracy': accuracy}

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Setting the training options and hyper-parameter settings. This is going to be the same across all experiments

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    weight_decay=0.001,
    lr_scheduler_type="linear",
    #logging_dir='./logs',
    #logging_steps=10,
    report_to=["none"],  # Disable W&B logging
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Please run this below cell when we want to train on all the data. Current setup is just to test if the training is working





In [ ]:
trainAllData = True
if trainAllData:
  small_train_dataset = train_dataset
  small_test_dataset = test_dataset

The Following section setup the different types of Models for training on the imdb dataset
# Base model with the classification head is finetuned.
The whole model is finetuned to imdb dataset

In [ ]:
# This will display the model structure and the layer structure
base_model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
trainer_base = Trainer(
    model=base_model,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=small_train_dataset,  # training dataset
    eval_dataset=small_test_dataset,   # evaluation dataset
    compute_metrics=compute_metrics,     # function for computing metrics
)

In [ ]:
trainer_base.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.252800,0.238788,0.930800
2,0.178600,0.204873,0.944000
3,0.123000,0.232998,0.941800
4,0.060500,0.274974,0.945280
5,0.039900,0.281288,0.947320


TrainOutput(global_step=7815, training_loss=0.13768309790281172, metrics={'train_runtime': 3430.0491, 'train_samples_per_second': 36.443, 'train_steps_per_second': 2.278, 'total_flos': 3.288888192e+16, 'train_loss': 0.13768309790281172, 'epoch': 5.0})

# Base model only the classification head is finetuned.
The classification head is finetuned to imdb dataset

In [ ]:
base_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
model = base_model
for param in model.base_model.parameters():
    param.requires_grad = False  # Freeze the encoder layers

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer_base_ch_only = Trainer(
    model=model,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=small_train_dataset,  # training dataset
    eval_dataset=small_test_dataset,   # evaluation dataset
    compute_metrics=compute_metrics,     # function for computing metrics
)

In [ ]:
trainer_base_ch_only.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.556100,0.504787,0.847560
2,0.453800,0.394157,0.861680
3,0.416900,0.358693,0.867280
4,0.412900,0.342835,0.870280
5,0.415500,0.338834,0.870760


TrainOutput(global_step=7815, training_loss=0.46770212438269754, metrics={'train_runtime': 1570.9524, 'train_samples_per_second': 79.57, 'train_steps_per_second': 4.975, 'total_flos': 3.288888192e+16, 'train_loss': 0.46770212438269754, 'epoch': 5.0})

# Custom model with no adapters used. All parameters fine tuned




In [ ]:
base_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
custom_model = ModifiedRobertaForSequenceClassification(base_model ,num_labels=2, adapter_hidden_dim=64, freeze_params = True, adapter_layers = [False] * 12)
# View the updated custom model. All encoder blocks should be similar and should have a pass through block instead of the adapter block
custom_model

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ModifiedRobertaForSequenceClassification(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (modified_layers): ModuleList(
    (0-11): 12 x ModifiedRobertaLayer(
      (adapter1): PassThroughBlock()
      (adapter2): PassThroughBlock()
      (attention): RobertaAttention(
        (self): RobertaSdpaSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=True)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): RobertaSelfOutput(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (LayerNorm): La

In [ ]:
trainer_custom_model = Trainer(
    model=custom_model,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=small_train_dataset,  # training dataset
    eval_dataset=small_test_dataset,   # evaluation dataset
    compute_metrics=compute_metrics,     # function for computing metrics
)

In [ ]:
trainer_custom_model.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.227100,0.223796,0.915520
2,0.180900,0.203923,0.940120
3,0.112100,0.250608,0.946800
4,0.052400,0.291552,0.945840
5,0.027900,0.295444,0.948920


TrainOutput(global_step=7815, training_loss=0.12754601096008653, metrics={'train_runtime': 3181.4214, 'train_samples_per_second': 39.291, 'train_steps_per_second': 2.456, 'total_flos': 0.0, 'train_loss': 0.12754601096008653, 'epoch': 5.0})

# Custom model with no adapters used. Classification head fine tuned

In [ ]:
# set-requires_grad will freeze all original layer parameters and only train the adapters and classification heads
base_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
custom_model = ModifiedRobertaForSequenceClassification(base_model ,num_labels=2, adapter_hidden_dim=64, freeze_params = True, adapter_layers = [False] * 12)
custom_model.set_requires_grad(False)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer_custom_model_ch_only = Trainer(
    model=custom_model,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=small_train_dataset,  # training dataset
    eval_dataset=small_test_dataset,   # evaluation dataset
    compute_metrics=compute_metrics,     # function for computing metrics
)

In [ ]:
trainer_custom_model_ch_only.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.662000,0.651069,0.685440
2,0.624900,0.613259,0.694520
3,0.604300,0.590812,0.719000
4,0.595700,0.581641,0.718360
5,0.591900,0.578416,0.719440


TrainOutput(global_step=7815, training_loss=0.6210606980766155, metrics={'train_runtime': 1553.7817, 'train_samples_per_second': 80.449, 'train_steps_per_second': 5.03, 'total_flos': 0.0, 'train_loss': 0.6210606980766155, 'epoch': 5.0})

# Custom model with all layers with adapters used. Adapters and Classification head fine tuned only

In [ ]:
base_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
model_all_adapters = ModifiedRobertaForSequenceClassification(base_model ,num_labels=2, adapter_hidden_dim=64, freeze_params = True, adapter_layers = [True] * 12)
# View the updated custom model. All encoder blocks should be similar and should have a pass through block instead of the adapter block
model_all_adapters.set_requires_grad(False)
model_all_adapters

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ModifiedRobertaForSequenceClassification(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (modified_layers): ModuleList(
    (0-11): 12 x ModifiedRobertaLayer(
      (adapter1): AdapterBlock(
        (fc1): Linear(in_features=768, out_features=64, bias=True)
        (relu): ReLU()
        (fc2): Linear(in_features=64, out_features=768, bias=True)
      )
      (adapter2): AdapterBlock(
        (fc1): Linear(in_features=768, out_features=64, bias=True)
        (relu): ReLU()
        (fc2): Linear(in_features=64, out_features=768, bias=True)
      )
      (attention): RobertaAttention(
        (self): RobertaSdpaSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): 

In [ ]:
trainer_model_all_adapters = Trainer(
    model=model_all_adapters,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=small_train_dataset,  # training dataset
    eval_dataset=small_test_dataset,   # evaluation dataset
    compute_metrics=compute_metrics,     # function for computing metrics
)

In [ ]:
trainer_model_all_adapters.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.694200,0.693791,0.500000
2,0.693600,0.694319,0.500000
3,0.693700,0.693473,0.500000
4,0.693400,0.693260,0.500000
5,0.693300,0.693147,0.500000


TrainOutput(global_step=7815, training_loss=0.693652753775042, metrics={'train_runtime': 2527.1325, 'train_samples_per_second': 49.463, 'train_steps_per_second': 3.092, 'total_flos': 0.0, 'train_loss': 0.693652753775042, 'epoch': 5.0})

# Custom model with lower layers(2) with adapters used. Adapters and Classification head fine tuned only




In [ ]:
adpt_lyrs = [False]*12
adpt_lyrs[0:2] = [True]*2
base_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
model_lower_adapters = ModifiedRobertaForSequenceClassification(base_model ,num_labels=2, adapter_hidden_dim=64, freeze_params = True, adapter_layers = adpt_lyrs)
model_lower_adapters.set_requires_grad(False)
# View the updated custom model. All encoder blocks should be similar and should have a pass through block instead of the adapter block
model_lower_adapters

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ModifiedRobertaForSequenceClassification(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (modified_layers): ModuleList(
    (0-1): 2 x ModifiedRobertaLayer(
      (adapter1): AdapterBlock(
        (fc1): Linear(in_features=768, out_features=64, bias=True)
        (relu): ReLU()
        (fc2): Linear(in_features=64, out_features=768, bias=True)
      )
      (adapter2): AdapterBlock(
        (fc1): Linear(in_features=768, out_features=64, bias=True)
        (relu): ReLU()
        (fc2): Linear(in_features=64, out_features=768, bias=True)
      )
      (attention): RobertaAttention(
        (self): RobertaSdpaSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Li

In [ ]:
trainer_model_lower_adapters = Trainer(
    model=model_lower_adapters,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=small_train_dataset,  # training dataset
    eval_dataset=small_test_dataset,   # evaluation dataset
    compute_metrics=compute_metrics,     # function for computing metrics
)

In [ ]:
trainer_model_lower_adapters.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.696500,0.693421,0.500000
2,0.695200,0.694697,0.500000
3,0.694000,0.693309,0.500000
4,0.694500,0.693336,0.500000
5,0.692500,0.643909,0.671040


TrainOutput(global_step=7815, training_loss=0.6935477697917917, metrics={'train_runtime': 2407.9395, 'train_samples_per_second': 51.912, 'train_steps_per_second': 3.246, 'total_flos': 0.0, 'train_loss': 0.6935477697917917, 'epoch': 5.0})

# Custom model with higher layers(2) with adapters used. Adapters and Classification head fine tuned only

In [ ]:
adpt_lyrs = [False]*12
adpt_lyrs[10:] = [True]*2
base_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
model_higher_adapters = ModifiedRobertaForSequenceClassification(base_model ,num_labels=2, adapter_hidden_dim=64, freeze_params = True, adapter_layers = adpt_lyrs)
model_higher_adapters.set_requires_grad(False)
# View the updated custom model. All encoder blocks should be similar and should have a pass through block instead of the adapter block
model_higher_adapters

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ModifiedRobertaForSequenceClassification(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (modified_layers): ModuleList(
    (0-9): 10 x ModifiedRobertaLayer(
      (adapter1): PassThroughBlock()
      (adapter2): PassThroughBlock()
      (attention): RobertaAttention(
        (self): RobertaSdpaSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=True)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): RobertaSelfOutput(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (LayerNorm): Lay

In [ ]:
trainer_model_higher_adapters = Trainer(
    model=model_higher_adapters,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=small_train_dataset,  # training dataset
    eval_dataset=small_test_dataset,   # evaluation dataset
    compute_metrics=compute_metrics,     # function for computing metrics
)

In [ ]:
trainer_model_higher_adapters.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.263500,0.231866,0.911720
2,0.234300,0.223885,0.922040
3,0.227800,0.189417,0.933040
4,0.206300,0.198605,0.934680
5,0.216900,0.191849,0.935600


TrainOutput(global_step=7815, training_loss=0.2584420456157155, metrics={'train_runtime': 1694.4101, 'train_samples_per_second': 73.772, 'train_steps_per_second': 4.612, 'total_flos': 0.0, 'train_loss': 0.2584420456157155, 'epoch': 5.0})

# Old section. Please ignore

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    weight_decay=0.001,
    lr_scheduler_type="linear",
    #logging_dir='./logs',
    #logging_steps=10,
    report_to=["none"],  # Disable W&B logging
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=train_dataset,  # training dataset
    eval_dataset=test_dataset,   # evaluation dataset
    compute_metrics=compute_metrics,     # function for computing metrics
)

Train the network for the given epochs

In [ ]:
#trainer.train()
#results = trainer.evaluate()
#print(results)


In [ ]:
os.getcwd()

'/content/drive/MyDrive/LearningDevelopment/Adarsh/OMSCS/c1DeepLearning/Final Project'

In [ ]:
# Save the model
#model.save_pretrained("./roberta_imdb_model")
#tokenizer.save_pretrained("./roberta_imdb_model")